In [1]:

# check for the GPU provided in the runtime
!nvidia-smi

Fri Nov  4 12:33:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   30C    P0    54W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  Off  | 00000000:0A:00.0 Off |                    0 |
| N/A   

In [2]:
# mostly pl is used while doing complex model training
import pytorch_lightning as pl
print(pl.__version__)

2022-11-04 12:33:56.128313: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-04 12:33:56.246630: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-04 12:33:56.781444: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-04 12:33:56.781520: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

1.2.6


In [3]:
# argparse makes it easier to write user friendly command line interfaces
import argparse
# package for faster file name matching
import glob
# makiing directories for data 
import os
# reading json files as the data is present in json files
import json
# time module for calculating the model runtime
import time
# Allows writing status messages to a file
import logging
# generate random float numbers uniformly
import random
# regex module for text 
import re
# module provides various functions which work on 
# iterators too produce complex iterators
from itertools import chain
from string import punctuation
import torch.nn.functional as F
# import torchtext
import pandas as pd
import torch
import numpy as np
from torch import nn
from torch.optim import Adam
# pandas for data manipulation
import pandas as pd
# numpy for array operations
import numpy as np
# PyTorch
import torch
# provides various classes representing file system paths
# with appropriate semantics
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
# import pytorch_lightning as pl

# splitting the data 
from sklearn.model_selection import train_test_split
# ANSII color formatting for ouput in terminal
from termcolor import colored
# wrapping paragraphs into string
import textwrap

from torch.optim import Adam

# model checkpoints in pretrained model
from pytorch_lightning.callbacks import ModelCheckpoint

'''
optimizer - AdamW
T5 Conditional Generator in which we'll give conditions
T5 tokenizer because it is fast
training the model without a learning rate
'''
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [4]:
# Seeds all the processes including numpy torch and other imported modules.
# pl.seed_everything(0)

In [5]:
# # check the version provided by Lightning
# import pytorch_lightning as pl
# print(pl.__version__)

In [6]:
# QA dataset from https://github.com/dmis-lab/bioasq-biobert
# which is in Zip format
# !gdown --id 1mxVUywvKzvA9bvrUc11RYuOTy7MYcXHF

In [7]:
# Unzipping the folder
# !unzip -q bio-QA.zip

In [8]:
df = pd.read_excel('ArithOpsTrain.xlsx',header=1)


In [9]:
df.shape

(979, 6)

In [10]:
# Dropping all the rows with repeated context and questions pairs.

# df = df.drop_duplicates(subset=["context"]).reset_index(drop=True)

In [11]:
df.shape

(979, 6)

In [12]:
df.head()

,Unnamed: 0,Description,Question,Equation,Input Numbers,Output
0,NaN,gino has number0 popsicle sticks . i have numb...,what is the sum of our popsicle sticks ?,+ number0 number1,63 50,113.0
1,NaN,lino picked up number0 shells at the seashore ...,how many shells did he pick up in all ?,+ number0 number1,292 324,616.0
2,NaN,there were number0 parents in the program and ...,how many people were present in the program ?,+ number0 number1,105 698,803.0
3,NaN,last saturday marie sold number0 magazines and...,what is the total number of reading materials ...,+ number0 number1,425 275,700.0
4,NaN,there are number0 birds on the fence . number1...,how many birds are on the fence ?,+ number0 number1,12 8,20.0


In [13]:
df.rename(columns = {'Description':'context','Question':'question','Equation':'answer_text'}, inplace = True)


In [14]:
df

,Unnamed: 0,context,question,answer_text,Input Numbers,Output
0,NaN,gino has number0 popsicle sticks . i have numb...,what is the sum of our popsicle sticks ?,+ number0 number1,63 50,113.0
1,NaN,lino picked up number0 shells at the seashore ...,how many shells did he pick up in all ?,+ number0 number1,292 324,616.0
2,NaN,there were number0 parents in the program and ...,how many people were present in the program ?,+ number0 number1,105 698,803.0
3,NaN,last saturday marie sold number0 magazines and...,what is the total number of reading materials ...,+ number0 number1,425 275,700.0
4,NaN,there are number0 birds on the fence . number1...,how many birds are on the fence ?,+ number0 number1,12 8,20.0
...,...,...,...,...,...,...
974,NaN,there are number0 bananas in janice 's banana ...,how big is each group ?,/ number0 number2,3300 5 75,44.0
975,NaN,janice has number0 bottle caps that must be pu...,how many bottle caps must go in each box ?,/ number0 number2,316 17 79,4.0
976,NaN,chris is inviting number0 friends to a party ....,how many cookies will each friend get ?,/ number1 number0,82 1804 10,22.0
977,NaN,keith has number0 marbles and number1 pencils ...,how many marbles does each friend get ?,/ number0 number2,5530 3 79,70.0


# Tokenization

In [15]:
# using the base T5 model having 222M params
MODEL_NAME ='t5-base'

In [16]:
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

In [17]:
new_tokens = ["number0",'number1','number2']

tokenizer.add_tokens(new_tokens,special_tokens=True)


3

In [18]:
sam=tokenizer('piyush number0')

In [19]:
sam['input_ids']

[2816, 63, 8489, 32100, 1]

In [20]:
sample_encoding = tokenizer('is the glass half empty or half full?', 'It depends on the initial state of the glass. If the glass starts out empty and liquid is added until it is half full, it is half full. If the glass starts out full and liquid is removed until it is half empty, it is half empty.')

In [21]:
sample_encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [22]:
print(sample_encoding["input_ids"])

[19, 8, 1905, 985, 6364, 42, 985, 423, 58, 1, 94, 5619, 30, 8, 2332, 538, 13, 8, 1905, 5, 156, 8, 1905, 3511, 91, 6364, 11, 4400, 19, 974, 552, 34, 19, 985, 423, 6, 34, 19, 985, 423, 5, 156, 8, 1905, 3511, 91, 423, 11, 4400, 19, 3641, 552, 34, 19, 985, 6364, 6, 34, 19, 985, 6364, 5, 1]


In [23]:

print(sample_encoding["attention_mask"])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [24]:
print(len(sample_encoding['input_ids']), len(sample_encoding['attention_mask']))

63 63


In [25]:
# Checking the decoding of the input ids

preds = [
         tokenizer.decode(input_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
         for input_id in sam['input_ids']
]

In [26]:
preds= " ".join(preds)
for wrap in textwrap.wrap(preds, width = 80):
  print(wrap)

pi y ush number0 </s>


In [27]:
sample_question = df.iloc[243]
sample_question

Unnamed: 0                                                     NaN
context          bryan had number0 precious stones in his colle...
question                   how much money did bryan get in total ?
answer_text                                      * number0 number1
Input Numbers                                            8 1785.00
Output                                                     14280.0
Name: 243, dtype: object

There exists a special seperator token in between the question and its answers.

Checking the encoding on the sample question

In [28]:
encoding = tokenizer(
    sample_question['question'],
    sample_question['context'],
    max_length=396,
    padding='max_length',
    truncation="only_second",
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)

In [29]:
encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [30]:
tokenizer.special_tokens_map

{'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '<pad>',
 'additional_special_tokens': "['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>', '<extra_id_46>', '<extra_id_47>', '<extra_id_48>', '<extra_id_49>', '<extra_id_50>', '<extra_id_51>', '<extra_id_52>', '<extra_i

In [31]:

tokenizer.eos_token, tokenizer.eos_token_id
# Input id of 1 represents end of sequence token.

('</s>', 1)

In [32]:
# Text representation pf the input ids

tokenizer.decode(encoding['input_ids'].squeeze())

'how much money did bryan get in total?</s> bryan had number0 precious stones in his collection which he sold to his friend from the jewelry store. if the stones were sold at $ number1 each</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <p

## Creating the labels for the answers

In [33]:
answer_encoding = tokenizer(
    sample_question['answer_text'],
    max_length=32,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)

In [34]:
tokenizer.decode(answer_encoding['input_ids'].squeeze())

'* number0 number1 </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

In [35]:
labels = answer_encoding["input_ids"]
labels

tensor([[ 1429, 32100, 32101,     1,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])

Labels after the end of sequence in the answer encoding has to be converted to -100 from 0 for the model evaluation.

In [36]:
labels[labels == 0] = -100

In [37]:
labels

tensor([[ 1429, 32100, 32101,     1,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100]])

## To create dataset

In [38]:
class BioQADataset(Dataset):
  def __init__(
      self,
      data:pd.DataFrame,
      tokenizer:T5Tokenizer,
      source_max_token_len: int = 396,
      target_max_token_len: int = 32,

      ):
    
    self.data =  data
    self.tokenizer =  tokenizer
    self.source_max_token_len =  source_max_token_len
    self.target_max_token_len =  target_max_token_len


  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    source_encoding = tokenizer(
      data_row['question'],
      data_row['context'],
      max_length=self.source_max_token_len,
      padding='max_length',
      
      
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    
    target_encoding = tokenizer(
      data_row['answer_text'],
      max_length=self.target_max_token_len,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    
    labels = target_encoding['input_ids']
    labels[labels==0] = -100

    return dict(
        question=data_row['question'],
        context=data_row['context'],
        answer_text=data_row['answer_text'],
        input_ids=source_encoding["input_ids"].flatten(),
        attention_mask=source_encoding['attention_mask'].flatten(),
        labels=labels.flatten()
    )




In [39]:
sample_dataset = BioQADataset(df, tokenizer)

In [40]:
for data in sample_dataset:
  print("Question: ", data['question'])
  print("Answer text: ", data['answer_text'])
  print("Input_ids: ", data['input_ids'][:10])
  print("Labels: ", data['labels'][:10])
  break

Question:  what is the sum of our popsicle sticks ?
Answer text:  + number0 number1
Input_ids:  tensor([ 125,   19,    8, 4505,   13,   69, 2783,    7,   23, 2482])
Labels:  tensor([ 1768, 32100, 32101,     1,  -100,  -100,  -100,  -100,  -100,  -100])


## Splitting into train and validation sets

In [41]:
train_df, val_df = train_test_split(df, test_size=0.05)

In [42]:
train_df.shape,  val_df.shape

((930, 6), (49, 6))

# Create pytorch lightning datamodule

In [43]:
class BioDataModule(nn.Module):
  def __init__(
      self,
      train_df: pd.DataFrame,
      test_df: pd.DataFrame,
      tokenizer:T5Tokenizer,
      batch_size: int = 8,
      source_max_token_len: int = 396,
      target_max_token_len: int = 32,
      ):
    super().__init__()
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def setup(self):
    self.train_dataset = BioQADataset(
        self.train_df,
        self.tokenizer,
        self.source_max_token_len,
        self.target_max_token_len
        )

    self.test_dataset = BioQADataset(
    self.test_df,
    self.tokenizer,
    self.source_max_token_len,
    self.target_max_token_len
    )
 
  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size=self.batch_size,
        shuffle=True,
        num_workers=4
        )
  def val_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size=self.batch_size,
        num_workers=4
        )

  def test_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size=1,
        num_workers=4
        )

In [44]:
BATCH_SIZE =4
N_EPOCHS = 6

data_module = BioDataModule(train_df, val_df, tokenizer, batch_size=BATCH_SIZE)
data_module.setup()

In [45]:
train_loader=data_module.train_dataloader()
valid_loader =data_module.val_dataloader()

In [46]:
for batch_idx, batch in enumerate(valid_loader):
    print(batch_idx)
    print(batch.keys())
    break

0
dict_keys(['question', 'context', 'answer_text', 'input_ids', 'attention_mask', 'labels'])


In [47]:
len(train_loader)

233

In [48]:
len(valid_loader)

13

## Loading and finetuning the T5-base model

## Building the PyTorch lightning module using T5ForConditionalGeneration model

In [49]:
class BioQAModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)


  def forward(self, input_ids, attention_mask, labels=None):
    output = self.model(
        input_ids, 
        attention_mask=attention_mask,
        labels=labels)

    return output.loss, output.logits


In [50]:
DEVICE = torch.device('cuda:4' if torch.cuda.is_available() else 'cpu')
model = BioQAModel() 
optim = torch.optim.AdamW(model.parameters(),lr=0.0001 )
model.to(DEVICE)


Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BioQAModel(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseReluDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): Linear(in_feature

In [51]:
next(model.parameters()).is_cuda


True

In [52]:
def generate_answer(question):
  source_encoding=tokenizer(
      question["question"],
      question['context'],
      max_length = 396,
      padding="max_length",
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"

  ).to(DEVICE)

  generated_ids = model.model.generate(
      input_ids=source_encoding["input_ids"],
      attention_mask=source_encoding["attention_mask"],
      num_beams=1,  # greedy search
      max_length=80,
      repetition_penalty=2.5,
      early_stopping=True,
      use_cache=True)
  
  preds = [
          tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
          for generated_id in generated_ids
  ]

  return "".join(preds)

In [53]:

 
 
def evaluate(expression,numbers_array):
    """
    Evaluate a given expression in prefix notation.
    Asserts that the given expression is valid.
    """
    stack = []
    sp=['+','-','*','/']

    # iterate over the string in reverse order
    for c in expression[::-1]:
 
        # push operand to stack
        if c in sp:
            # pop values from stack can calculate the result
            # push the result onto the stack again
            if len(stack)<2:
                return 0,-1
            o1 = stack.pop()
            o2 = stack.pop()
 
            if c == '+':
                stack.append(o1 + o2)
 
            elif c == '-':
                stack.append(o1 - o2)
 
            elif c == '*':
                stack.append(o1 * o2)
 
            elif c == '/':
                stack.append(o1 / o2)
        else:
            no_index=c[-1]
            if(no_index.isdigit() and int(no_index)<len(numbers_array)) :
                stack.append(float(numbers_array[int(no_index)]))
    if len(stack)!=1:
        return 0,-1
    return 1,stack.pop()

In [54]:
def infer(dataframe):
    count=0
    for index in range(dataframe.shape[0]):
        # if count!=index:
        #     print(index)
        sample_quest = dataframe.iloc[index]
        truth=sample_quest["answer_text"].split()
        # print(truth)
        no=sample_quest['Input Numbers'].split()
        # print(no)
        predicted=generate_answer(sample_quest).split()
        actual=sample_quest['Output']
        # print(predicted)
        bool1,val1=evaluate(truth,no)
        bool2,val2=evaluate(predicted,no)
        # print(bool1,val1)
        # print(bool2,val2)
        if(bool2==1):
            if(round(val2,2)==round(actual,2)):
                count=count+1
            # else:
            #     print(index)
    return count/dataframe.shape[0]


In [55]:
def validation_loss(model, data_loader, device):
    loss_val=0
    with torch.no_grad():

        # correct_pred, num_examples = 0, 0

        for batch_idx, batch in enumerate(data_loader):

            ### Prepare data
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            loss,outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            # loss, logits = outputs['loss'], outputs['logits']
            loss_val=loss_val+loss
            # _, predicted_labels = torch.max(logits, 1)

            # num_examples += labels.size(0)

            # correct_pred += (predicted_labels == labels).sum()
    return loss_val/len(data_loader)

In [56]:
from sklearn.utils import validation


NUM_EPOCHS=50

start_time = time.time()

for epoch in range(NUM_EPOCHS):
    print("########################################################################################")
    print("current Epoch is ", epoch+1)
    model.train()
    curr_loss=0
    for batch_idx, batch in enumerate(train_loader):
        
        ### Prepare data
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        ### Forward
        loss,outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        # loss, logits = outputs['loss'], outputs['logits']
        curr_loss =curr_loss+loss
        ### Backward
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        ### Logging
        # if not batch_idx % 250:
        #     print (f'Epoch: {epoch+1:04d}/{NUM_EPOCHS:04d} | '
        #            f'Batch {batch_idx:04d}/{len(train_loader):04d} | '
        #            f'Loss: {loss:.4f}')



    print("training loss ",curr_loss/len(train_loader))  

         
    model.eval()

    with torch.set_grad_enabled(False):
        print("validation Loss",validation_loss(model, valid_loader, DEVICE))
        print("validation Accuracy",infer(val_df))
        if epoch%8==0:
            print("The Training accuracy",infer(train_df))
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
# print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

########################################################################################
current Epoch is  1
training loss  tensor(4.5310, device='cuda:4', grad_fn=<DivBackward0>)
validation Loss tensor(2.6238, device='cuda:4')
validation Accuracy 0.3673469387755102
The Training accuracy 0.4096774193548387
Time elapsed: 1.61 min
########################################################################################
current Epoch is  2
training loss  tensor(2.2388, device='cuda:4', grad_fn=<DivBackward0>)
validation Loss tensor(1.8178, device='cuda:4')
validation Accuracy 0.42857142857142855
Time elapsed: 2.16 min
########################################################################################
current Epoch is  3
training loss  tensor(1.7143, device='cuda:4', grad_fn=<DivBackward0>)
validation Loss tensor(1.2553, device='cuda:4')
validation Accuracy 0.4897959183673469
Time elapsed: 2.72 min
########################################################################################

In [ ]:
len(train_loader)

59

: 

: 

: 

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')

: 

: 

: 

In [ ]:
model = BioQAModel() 
# optim = torch.optim.AdamW(model.parameters(),lr=0.0001 )
model.to(DEVICE) # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('/data1/home/piyushmishra/DLNLP/model_weights.pth'))
model.eval()

Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BioQAModel(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseReluDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): Linear(in_feature

: 

: 

: 

## Generate answers for the questions in the validation set

In [ ]:
def infer2(dataframe):
    count=0
    output=[]
    trueoutput=[]
    for index in range(dataframe.shape[0]):
        # if count!=index:
        #     print(index)
        sample_quest = dataframe.iloc[index]
        truth=sample_quest["answer_text"].split()
        # print(truth)
        no=sample_quest['Input Numbers'].split()
        # print(no)
        predicted=generate_answer(sample_quest).split()
        actual=sample_quest['Output']
        trueoutput.append(actual)
        # print(predicted)
        # bool1,val1=evaluate(truth,no)
        bool2,val2=evaluate(predicted,no)
        output.append(val2)

        # print(bool1,val1)
        # print(bool2,val2)
        if(bool2==1):
            if(round(val2,2)==round(actual,2)):
                count=count+1
            # else:
            #     print(index)
    return count/dataframe.shape[0],trueoutput,output


: 

: 

: 

In [ ]:
a,b,c=infer2(val_df)

: 

: 

: 

In [ ]:
a

0.6122448979591837

: 

: 

: 

In [ ]:
for i in range(len(b)):
    print(b[i],c[i])

14.0 14.0
30.0 25.0
19.0 19.0
90.0 225.0
7.0 27.0
336.0 1344.0
3.0 3.0
51.0 51.0
52.0 4.0
7.0 7.0
208.0 208.0
26.0 26.0
20.0 20.0
9.0 3.0
6.0 6.0
11.0 132.0
300.0 300.0
42.0 42.0
35.0 35.0
13.0 -13.0
20.0 20.0
41.0 32.0
203.0 203.0
43.07 43.07
38.0 110.0
6.0 150.0
20.0 20.0
2448.0 2448.0
68.0 -12.0
315.0 315.0
14.0 14.0
14.0 6.0
612.0 612.0
45552.0 45552.0
10.0 10.0
40.0 67.0
60.0 3.0
952.0 952.0
3.0 23.0
34.0 34.0
136.0 136.0
22.0 65.0
61.0 61.0
1.0 -17.0
23.0 23.0
96.0 96.0
39.0 39.0
34.0 6272.0
8.0 8.0


: 

: 

: 

In [ ]:
def infer3(dataframe):
    output=[]
    for index in range(dataframe.shape[0]):
        # if count!=index:
        #     print(index)
        sample_quest = dataframe.iloc[index]
        # print(truth)
        no=sample_quest['Input Numbers'].split()
        # print(no)
        predicted=generate_answer(sample_quest).split()
        # print(predicted)
        # bool1,val1=evaluate(truth,no)
        bool2,val2=evaluate(predicted,no)
        output.append(val2)

        # print(bool1,val1)
        # print(bool2,val2)
       
            # else:
            #     print(index)
    return output

: 

: 

: 

In [ ]:
testdf = pd.read_excel('ArithOpsTestData1.xlsx')
testdf.rename(columns = {'Description':'context','Question':'question'}, inplace = True)


: 

: 

: 

In [ ]:
testdf.head()

,context,question,Input Numbers
0,number0 red apples and number1 green apples ar...,how many apples are in the basket ?,7 2
1,ellen has number0 more balls than marin . mari...,how many balls does ellen have ?,6 9
2,janet has number0 oranges and sharon has numbe...,how many oranges do janet and sharon have toge...,9 7
3,allan brought number0 balloons and jake brough...,how many balloons did allan and jake have in t...,2 4
4,adam has number0 more apples than jackie . jac...,how many apples does adam have ?,5 9


: 

: 

: 

In [ ]:
final_output=infer3(testdf)

: 

: 

: 

In [ ]:
final_output

[9.0,
 -3.0,
 16.0,
 6.0,
 -4.0,
 -3.0,
 8.0,
 -4.0,
 18.0,
 11.0,
 24.0,
 19.0,
 45.0,
 2.0,
 8.0,
 -7.0,
 -3.0,
 -7.0,
 -12.0,
 6.0,
 5.0,
 -4.0,
 3.0,
 1.0,
 5.0,
 1.0,
 3.0,
 38.0,
 38.0,
 13.0,
 46.0,
 11.0,
 7.0,
 5.0,
 6.0,
 4.0,
 6.0,
 11.0,
 7.0,
 18.0,
 7.0,
 7.0,
 7.0,
 5.0,
 28.0,
 8.0,
 38.0,
 929.0,
 62.0,
 46.0,
 20.0,
 79.0,
 67.0,
 14.0,
 9.0,
 -6.0,
 -23.0,
 17.0,
 -32.0,
 22.0,
 42.0,
 35.0,
 19.0,
 128.0,
 57.0,
 286.0,
 41.0,
 0.75,
 300.0,
 10.0,
 58.0,
 4.0,
 21.0,
 0.1111111111111111,
 20.0,
 15.0,
 115.0,
 24.0,
 0.06666666666666667,
 190.0,
 311.0,
 164.0,
 117.0,
 103.0,
 139.0,
 352.0,
 -15.0,
 -188.0,
 100.0,
 55.0,
 82.0,
 828.0,
 12.0,
 0.3333333333333333,
 45.0,
 40.0,
 54.0,
 56.0,
 12.0,
 8.0,
 30.0,
 72.0,
 3285.0,
 10.0,
 0.08333333333333333,
 7.0,
 9.0,
 60.0,
 0.16666666666666666,
 0.25,
 20.0,
 0.14285714285714285,
 5.0,
 0.1111111111111111,
 432.0,
 6003.0,
 2250.0,
 20.0,
 7.0,
 2205.0,
 486.0,
 372.0,
 0.2222222222222222,
 1035.0,
 -7.0,
 40.0,

: 

: 

: 

In [ ]:
import pandas as pd
import openpyxl

df = pd.DataFrame(final_output,columns=['Piyush Kumar Mishra'])


: 

: 

: 

In [ ]:
df

,Piyush Kumar Mishra
0,9.0
1,-3.0
2,16.0
3,6.0
4,-4.0
...,...
233,2.8
234,12.0
235,600.0
236,76.0


: 

: 

: 

In [ ]:
df.to_excel('Piyush3.xlsx', index=False, header=True)

: 

: 

: 

In [ ]:
df2

,Piyush Kumar Mishra
0,11
1,21
2,31


: 

: 

: 

In [ ]:
import pandas as pd
df2 = pd.read_excel('/data1/home/piyushmishra/DLNLP/Assignment_5b/Piyush3.xlsx')
df3=pd.read_excel('/data1/home/piyushmishra/DLNLP/Assignment_5b/ArithOpsTestData1.xlsx')


: 

: 

: 

In [ ]:
df2.iloc[1]

Piyush Kumar Mishra    15.0
Name: 1, dtype: float64

: 

: 

: 

In [ ]:
count=0
for i in range(df3.shape[0]):
    if(df2.iloc[i]['Piyush Kumar Mishra']==df3.iloc[i]['Output']):
        count=count+1

: 

: 

: 

In [ ]:
count/df3.shape[0]

0.5546218487394958

: 

: 

: 

In [ ]:
sample_quest = train_df.iloc[0]
truth=sample_quest["answer_text"].split()
print(truth)
no=sample_quest['Input Numbers'].split()
print(no)
predicted=generate_answer(sample_quest).split()
actual=sample_quest['Output']
print(predicted)
bool1,val1=evaluate(truth,no)
bool2,val2=evaluate(predicted,no)

['+', 'number0', 'number1']
['35', '56']
['+', 'number0', 'number1']


: 

: 

: 

In [ ]:
predicted[3][-1].isdigit()

IndexError: list index out of range

: 

: 

: 